In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import mysql.connector
import ipynb
from ipynb.fs.full.SqlQuery import brandAdd
from ipynb.fs.full.SqlQuery import categorieAdd
from ipynb.fs.full.SqlQuery import AddProduct
from ipynb.fs.full.SqlQuery import getCagId
from ipynb.fs.full.SqlQuery import getBrandId
from ipynb.fs.full.SqlQuery import AddProductAttribut
from ipynb.fs.full.SqlQuery import AddLocalStoreProduct
from ipynb.fs.full.SqlQuery import Addlocalstors

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)

In [3]:
def sbs_links():
  sbs_list=[]
  web = requests.get('https://www.sbsinformatique.com/')
  parse = BeautifulSoup(web.content, 'html.parser')
  sbs_links= parse.find_all('a', class_="fontcustom2")
  for sbs in sbs_links:
    link=sbs['href']
    info=sbs.span.text
    if(not(info == 'PC BUILDER')):
      sbs_dict={
          'name':info,
          'link':link
      }
      sbs_list.append(sbs_dict)
  return sbs_list

In [7]:
def sbs_details(link):
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    try:
        uid= parse.find('p', class_='reference').text
    except:
        uid='Undfined'
    try:
        stock= parse.find('span', id='product-availability').text.replace('\ue5ca','').strip()
    except:
        stock='non disonible'
    try:
        brand = parse.find('section', class_='page-content').img['src']
    except:
        brand='Unknown'
    try:
        des = parse.find('div', class_='product-desc').p.text
    except:
        des='None'
    label, data='',''
    name= parse.find_all('dt', class_='name')
    value = parse.find_all('dd', class_='value')
    fich_list=[]
    for n in name:
        fich_tech={
            'label': n.text,
            'data': ''
            }
        fich_list.append(fich_tech)
    i=0
    for v in value:
        fich_list[i]['data']=v.text
        i +=1
    details={
      'uid':uid,
      'stock':stock,
      'brand':brand,
      'des':des,
      'fich_tech':fich_list
    }
    return details

In [8]:
print(sbs_details('https://www.sbsinformatique.com/pcs-gamers-tunisie/pc-gamer-vento-ryzen-5-3400g-8gb-tunisie.html'))

{'uid': ' Référence: VENTO-RYZEN-5-3400G-8 ', 'stock': 'Disponible', 'brand': 'https://www.sbsinformatique.com/4223-large_default/pc-gamer-vento-ryzen-5-3400g-8gb-tunisie.jpg', 'des': 'Processeur: Processeur AMD Ryzen 5 3400G - Carte mère: MSI A320M-A PRO MAX - Disque Dur: SSD HP S600 2.5 240GB - Barrettes mémoires: 8GB DDR4 3200 MHZ- BOITIER: FSP VENTO VG08FE - Alimentation: FSP 500-60AHBC 500W CERTIFIEE 85', 'fich_tech': [{'label': 'PROCESSEUR', 'data': 'AMD Ryzen 5 3400G'}, {'label': 'CHIPSET GRAPHIQUE', 'data': 'AMD Radeon Vega 11 Graphics'}, {'label': 'TAILLE DE LA MÉMOIRE', 'data': '8 Go'}]}


In [5]:
def next_url(url, x):
  if(x == 1):
    url=url+'?page='+str(x)
  else:
    url=url[:-1]
    if(x>10):
      url=url[:-1]
    url=url+str(x)
  web = requests.get(url)
  parse = BeautifulSoup(web.content, 'html.parser')
  if(parse.find('section', class_='page-content page-not-found')):
    return False  

  return url

In [9]:
supCatId=0
list = sbs_links()
for l in list:
    info = l['name']
    link = l['link']
    categorieAdd(info, link, 19)
    supCatId=getCagId(info,19)
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    for x in range(1, 29):
        link= next_url(link,x)
        print(link)
        if(not(link == False)):
            web = requests.get(link)
            parse = BeautifulSoup(web.content, 'html.parser')
            card= parse.find_all('div', class_='item-product col-xs-12 col-sm-6 col-md-6 col-lg-4')
            for c in card:
                name= c.find('a', class_='product_name').text
                img= c.find('a', class_='thumbnail product-thumbnail').img['src']
                price= c.find('div', class_='product-price-and-shipping').span.text.replace('&nbsp;','')
                det_link= c.find('a', class_='product_name')['href']
                det=sbs_details(det_link)
                brandAdd(det['brand'], det['brand'], 19)
                supBrandId=getBrandId(det['brand'], 19)
                prodId=AddProduct(name, det_link, det['uid'], det['des'], price, det['stock'], 19, supBrandId, supCatId, img)
                if(det['fich_tech']):
                    AddProductAttribut(det['fich_tech'], prodId)
        else:
            break  

https://www.sbsinformatique.com/pcs-gamers-tunisie/?page=1
False
https://www.sbsinformatique.com/stations-pro-tunisie/?page=1
https://www.sbsinformatique.com/stations-pro-tunisie/?page=2
https://www.sbsinformatique.com/stations-pro-tunisie/?page=3
https://www.sbsinformatique.com/stations-pro-tunisie/?page=4
https://www.sbsinformatique.com/stations-pro-tunisie/?page=5
https://www.sbsinformatique.com/stations-pro-tunisie/?page=6
https://www.sbsinformatique.com/stations-pro-tunisie/?page=7
https://www.sbsinformatique.com/stations-pro-tunisie/?page=8
https://www.sbsinformatique.com/stations-pro-tunisie/?page=9
https://www.sbsinformatique.com/stations-pro-tunisie/?page=10
https://www.sbsinformatique.com/stations-pro-tunisie/?page=11
https://www.sbsinformatique.com/stations-pro-tunisie/?page=12
https://www.sbsinformatique.com/stations-pro-tunisie/?page=13
https://www.sbsinformatique.com/stations-pro-tunisie/?page=14
https://www.sbsinformatique.com/stations-pro-tunisie/?page=15
https://www.sb